# Cleanup

> *This notebook should work well in the `Python 3 (Data Science)` kernel in SageMaker Studio, or `conda_python3` in SageMaker Notebook Instances*

If you ran this Forecast POC in your own account and would like to avoid ongoing charges, run the commands below to cleanup the resources created in the previous notebooks, including:

* S3 Bucket
* IAM Role
* Dataset Group
* Datasets
* Import Jobs
* Predictors
* Forecasts
* Forecast Export Jobs


## Pre-requisites

The cell below will import necessary libraries, retrieve stored variables from previous notebooks, and connect to Forecast, S3, and IAM via the Boto3 Python library.

In [1]:
# Python Built-Ins:
from time import sleep

# External Dependencies:
import boto3
import botocore.exceptions

# Local Dependencies:
import util

# Retrieve stored variables
%store -r

# Create connections to Forecast, S3, and IAM
session = boto3.Session(region_name=region) 
forecast = session.client("forecast")
iam = session.client("iam")
s3 = session.client("s3")


## Define Objects to Delete

The resources we created have to be deleted in a specific order. The cell below will build a recursive inventory of the child objects belonging to our top-level Dataset Group object. We'll then print out everything we plan to delete. Make sure to review this output carefully to ensure it matches up with what you expect to delete.

In [ ]:
# Build inventory of child objects related to DatasetGroup to delete...
datasetGroupResponse=forecast.describe_dataset_group(DatasetGroupArn=datasetGroupArn)

# Get Datasets belonging to the parent Dataset Group
datasetArns=datasetGroupResponse['DatasetArns']

# Get Import Jobs belonging to Datasets
importJobArns=[]
for d in datasetArns:
    importJobResponse=forecast.list_dataset_import_jobs(Filters=[
        {
            'Key': 'DatasetArn',
            'Value': d,
            'Condition': 'IS' 
        }
    ])
    if len(importJobResponse['DatasetImportJobs']) > 0:
        importJobArns.append(importJobResponse['DatasetImportJobs'][0]['DatasetImportJobArn'])

# Get Predictors belonging to Dataset Group
predictorArns=[]
predictorResponse=forecast.list_predictors(Filters=[
        {
            'Key': 'DatasetGroupArn',
            'Value': datasetGroupArn,
            'Condition': 'IS'
        }
    ])

for p in predictorResponse['Predictors']:
    predictorArns.append(p['PredictorArn'])

# Get Forecasts belonging to Dataset Group
forecastArns=[]
forecastResponse=forecast.list_forecasts(Filters=[
        {
            'Key': 'DatasetGroupArn',
            'Value': datasetGroupArn,
            'Condition': 'IS'
        }
    ])

for f in forecastResponse['Forecasts']:
    forecastArns.append(f['ForecastArn'])
    
# Get Forecast Export Jobs
forecastExportJobArns=[]
forecastExportJobResponse=forecast.list_forecast_export_jobs()
for fa in forecastArns:
    forecastName=fa.split('/')[-1]
    for e in forecastExportJobResponse['ForecastExportJobs']:
        if forecastName in e['ForecastExportJobArn']:
            forecastExportJobArns.append(e['ForecastExportJobArn'])

bucket_names = [bucket_name]
if bucket_name != export_bucket_name:
    bucket_names.append(export_bucket_name)

# Print list of resources that will be deleted
print("The following resources will be deleted if you execute the next cell...")
print("")
print("S3 Bucket(s): ")
for b in bucket_names:
    print(b)
print("")
print("IAM Role: ")
print(forecast_role_arn)
print("")
print("Dataset Group: ")
print(datasetGroupArn)
print("")
print("Datasets: ")
for d in datasetArns:
    print(d)
print("")
print("Import Jobs: ")
for i in importJobArns:
    print(i)
print("")
print("Predictors: ")
for p in predictorArns:
    print(p)
print("")
print("Forecasts: ")
for f in forecastArns:
    print(f)
print("")
print("Forecast Export Jobs: ")
for e in forecastExportJobArns:
    print(e)
print("")


## Delete Forecast Objects

Once you've verified that the inventory of resources listed above is safe to delete, run the cell below to delete the Forecast resources. This cell will run the delete commands asyncronously and will print "Successful delete" for each object it deletes. The full deletion typically takes around 15 minutes.

In [ ]:
# Delete the Forecast Export Jobs:
for e in forecastExportJobArns:
    util.wait_till_delete(lambda: forecast.delete_forecast_export_job(ForecastExportJobArn=e))

# Delete the Forecasts
for f in forecastArns:
    util.wait_till_delete(lambda: forecast.delete_forecast(ForecastArn=f))

# Delete the Predictors
for p in predictorArns:
    util.wait_till_delete(lambda: forecast.delete_predictor(PredictorArn=p))

# Delete the Import Jobs
for i in importJobArns:
    util.wait_till_delete(lambda: forecast.delete_dataset_import_job(DatasetImportJobArn=i))

# Delete the Datasets
for d in datasetArns:
    util.wait_till_delete(lambda: forecast.delete_dataset(DatasetArn=d))

# Delete the Dataset Group
util.wait_till_delete(lambda: forecast.delete_dataset_group(DatasetGroupArn=datasetGroupArn))


## Confirm Forecast Objects are Deleted

We'll know the cell above is done asychronously deleting child objects once the top-level Dataset Group parent object is deleted. Run the cell below, and wait until you get a "Forecast objects succesfully deleted" message before proceeding further.

In [ ]:
# Loop on describe_dataset_group
while True:
    try:
        responseStatus=forecast.describe_dataset_group(DatasetGroupArn=datasetGroupArn)['Status']
        print(responseStatus)
        sleep(10)
    except botocore.exceptions.ClientError as e:
        # When given the resource not found exception, deletion has occured
        if e.response['Error']['Code'] == 'ResourceNotFoundException':
            print('Forecast objects succesfully deleted')
            break


## Delete IAM and S3 Objects

Now that the Forecast objects are cleaned up, we can delete the rest of the IAM and S3 resources we created.

In [ ]:
forecast_role_name = role_arn.split("/")[-1]

# Detach Role Policies from the IAM Role
# NOTE: Doesn't delete policies, if you created any custom ones!
attached_policies_response = iam.list_attached_role_policies(RoleName=forecast_role_name)
for policy in attached_policies_response["AttachedPolicies"]:
    iam.detach_role_policy(RoleName=forecast_role_name, PolicyArn=policy["PolicyArn"])

# Delete the IAM Role    
iam.delete_role(RoleName=forecast_role_name)

In [ ]:
for b in bucket_names:
    # Delete all objects from the S3 Bucket
    bucket = session.resource("s3").Bucket(b)
    bucket.objects.all().delete()
    sleep(5)

    # Delete the S3 Bucket
    s3.delete_bucket(Bucket=b)

## Last Steps

Congratulations! You've successfully cleaned up the default POC objects from your account. If you created additional Dataset Groups or other resources that were not included in the default POC, be sure to clean those up manually or modify the scripts above to clean them up for you.

The final step is to go back to the CloudFormation console and delete the Stack you provisioned to create these notebooks.

We hope you enjoyed learning how to predict the future with AWS Forecast.